In [2]:
use std::thread;
use std::time::Duration;

fn simulated_expensive_calculation(intensity: u32) -> u32 {
  println!("calculating slowly...");
  thread::sleep(Duration::from_secs(2));
  intensity
}

In [3]:
simulated_expensive_calculation(5)

calculating slowly...


5

In [4]:
// rough implementation
fn generate_workout(intensity: u32, random_number: u32) {
  if intensity < 25 {
    println!(
      "Today, do {} pushups!",
      simulated_expensive_calculation(intensity)
    );
    println!(
      "Next, do {} situps!",
      simulated_expensive_calculation(intensity)
    );
  } else {
    if random_number == 3 {
      println!("Take a break today!");
    } else {
      println!(
        "Today, run for {} minutes!",
        simulated_expensive_calculation(intensity)
      );
    }
  }
}

In [5]:
// refacter by binding expensive result
fn generate_workout(intensity: u32, random_number: u32) {
  let expensive_result = simulated_expensive_calculation(intensity);
  if intensity < 25 {
    println!(
      "Today, do {} pushups!",
      expensive_result
    );
    println!(
      "Next, do {} situps!",
      expensive_result
    );
  } else {
    if random_number == 3 {
      // expensive_result would be redundant if context would match this case
      println!("Take a break today!");
    } else {
      println!(
        "Today, run for {} minutes!",
        expensive_result
      );
    }
  }
}

In [6]:
// refacter using closuer
fn generate_workout(intensity: u32, random_number: u32) {
  let expensive_closure = |num| {
    println!("calculating slowly...");
    thread::sleep(Duration::from_secs(2));
    num
  }

  if intensity < 25 {
    println!(
      "Today, do {} pushups!",
      expensive_closure(intensity)
    );
    // Run closure again, but implove later
    println!(
      "Next, do {} situps!",
      expensive_closure(intensity)
    );
  } else {
    if random_number == 3 {
      println!("Take a break today!");
    } else {
      println!(
        "Today, run for {} minutes!",
        expensive_closure(intensity)
      );
    }
  }
}

Error: expected `;`, found keyword `if`

In [7]:
// Rust infer type from context, but you can't build if Rust can't infer the type
{
  let expensive_closure = |num| {
    println!("calculating slowly...");
    thread::sleep(Duration::from_secs(2));
    num
  };
}

Error: type annotations needed

In [8]:
// If it is not inferable, you need to annotate type implicitly.
{
  let expensive_closure = |num: u32| -> u32 {
    println!("calculating slowly...");
    thread::sleep(Duration::from_secs(2));
    num
  };
}

()

In [9]:
{
  let example_closure = |x| x;
  let s = example_closure(String::from("hello"));
  s
}

"hello"

In [10]:
{
  let example_closure = |x| x;
  let n = example_closure(5);
  n
}

5

In [11]:
// If you invoke closure with different type interface, Rust compiler tell you build error.
{
  let example_closure = |x| x;
  let s = example_closure(String::from("hello"));
  let n = example_closure(5);
  (s, n)
}

Error: mismatched types

In [ ]:
struct Catcher<T> where T: Fn(u32) -> u32 {
  calculation: T,
  value: Option<u32>,
}

In [ ]:
impl<T> Catcher<T>
  where T: Fn(u32) -> u32 
{
  fn new(calculation: T) -> Catcher<T> {
    Catcher {
      calculation,
      value: None,
    }
  }
  fn value(&mut self, arg: u32) -> u32 {
    match self.value {
      Some(v) => v,
      None => {
        let v = (self.calculation)(arg);
        self.value = Some(v);
        v
      }
    }
  }
}

In [ ]:
// refacter using closuer and catcher
fn generate_workout(intensity: u32, random_number: u32) {
  let mut expensive_closure = Catcher::new(|num| {
    println!("calculating slowly...");
    thread::sleep(Duration::from_secs(2));
    num
  });

  if intensity < 25 {
    println!(
      "Today, do {} pushups!",
      expensive_closure.value(intensity)
    );
    println!(
      "Next, do {} situps!",
      expensive_closure.value(intensity)
    );
  } else {
    if random_number == 3 {
      println!("Take a break today!");
    } else {
      println!(
        "Today, run for {} minutes!",
        expensive_closure.value(intensity)
      );
    }
  }
}

In [ ]:
generate_workout(10, 7)

calculating slowly...
Today, do 10 pushups!
Next, do 10 situps!


()

In [ ]:
// Current catcher only accepts same argument.
fn call_with_different_values() {
  let mut c = Catcher::new(|a| a);

  let v1 = c.value(1);
  let v2 = c.value(2);

  assert_eq!(v2, 2);
}

In [ ]:
call_with_different_values()

thread '<unnamed>' panicked at 'assertion failed: `(left == right)`
  left: `1`,
 right: `2`', src/lib.rs:8:3
stack backtrace:
   0: _rust_begin_unwind
   1: std::panicking::begin_panic_fmt
   2: _run_user_code_18
   3: evcxr::runtime::Runtime::run_loop
   4: evcxr::runtime::runtime_hook
   5: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [ ]:
use std::collections::HashMap;

struct Catcher2<T> where T: Fn(u32) -> u32 {
  calculation: T,
  values: HashMap<u32, u32>,
}

In [ ]:
use std::collections::HashMap;

impl<T> Catcher2<T>
  where T: Fn(u32) -> u32 
{
  fn new(calculation: T) -> Catcher2<T> {
    Catcher2 {
      calculation,
      values: HashMap::new(),
    }
  }
  fn value(&mut self, arg: u32) -> u32 {
    match self.values.get(&arg) {
      Some(v) => *v,
      None => {
        let v = (self.calculation)(arg);
        self.values.insert(arg, v);
        v
      }
    }
  }
}

In [ ]:
fn call_with_different_values() {
  let mut c = Catcher2::new(|a| a);

  let v1 = c.value(1);
  let v2 = c.value(2);

  assert_eq!(v2, 2);
}

In [ ]:
call_with_different_values()

()

In [ ]:
{
  fn generate_workout(intensity: u32, random_number: u32) {
    let mut expensive_closure = Catcher2::new(|num| {
      println!("calculating slowly...");
      thread::sleep(Duration::from_secs(2));
      num
    });

    if intensity < 25 {
      println!(
        "Today, do {} pushups!",
        expensive_closure.value(intensity)
      );
      println!(
        "Next, do {} situps!",
        expensive_closure.value(intensity)
      );
    } else {
      if random_number == 3 {
        println!("Take a break today!");
      } else {
        println!(
          "Today, run for {} minutes!",
          expensive_closure.value(intensity)
        );
      }
    }
  }
  generate_workout(10, 3)
}

calculating slowly...
Today, do 10 pushups!
Next, do 10 situps!


()

In [ ]:
use std::collections::HashMap;

struct Catcher3<T> where T: Fn(u32) -> u32 {
  calculation: T,
  values: HashMap<u32, u32>,
}

In [ ]:
use std::collections::HashMap;

impl<T> Catcher3<T>
  where T: Fn(u32) -> u32 
{
  fn new(calculation: T) -> Catcher3<T> {
    Catcher3 {
      calculation,
      values: HashMap::new(),
    }
  }
  fn value(&mut self, arg: u32) -> &u32 {
    // NOET: self.calculation will be invoked even self.values.entry return Some.
    self.values.entry(arg).or_insert((self.calculation)(arg))
  }
}

In [ ]:
{
  fn generate_workout(intensity: u32, random_number: u32) {
    let mut expensive_closure = Catcher3::new(|num| {
      println!("calculating slowly...");
      thread::sleep(Duration::from_secs(2));
      num
    });

    if intensity < 25 {
      println!(
        "Today, do {} pushups!",
        expensive_closure.value(intensity)
      );
      println!(
        "Next, do {} situps!",
        expensive_closure.value(intensity)
      );
    } else {
      if random_number == 3 {
        println!("Take a break today!");
      } else {
        println!(
          "Today, run for {} minutes!",
          expensive_closure.value(intensity)
        );
      }
    }
  }
  generate_workout(10, 3)
}

calculating slowly...
Today, do 10 pushups!
calculating slowly...
Next, do 10 situps!


()

In [13]:
use std::collections::HashMap;

struct Catcher4<T, U, V> where T: Fn(U) -> V {
  calculation: T,
  values: HashMap<U, V>,
}

In [18]:
use std::collections::HashMap;

impl<T, U, V> Catcher4<T, U, V>
  where 
    T: Fn(U) -> V,
    U: std::hash::Hash + Eq + Copy,
    V: Copy,
{
  fn new(calculation: T) -> Catcher4<T, U, V> {
    Catcher4 {
      calculation,
      values: HashMap::new(),
    }
  }
  fn value(&mut self, arg: U) -> V {
    match self.values.get(&arg) {
      Some(v) => *v,
      None => {
        let v = (self.calculation)(arg);
        self.values.insert(arg, v);
        v
      }
    }
  }
}

In [19]:
{
  fn generate_workout(intensity: u32, random_number: u32) {
    let mut expensive_closure = Catcher4::new(|num| {
      println!("calculating slowly...");
      thread::sleep(Duration::from_secs(2));
      num
    });

    if intensity < 25 {
      println!(
        "Today, do {} pushups!",
        expensive_closure.value(intensity)
      );
      println!(
        "Next, do {} situps!",
        expensive_closure.value(intensity)
      );
    } else {
      if random_number == 3 {
        println!("Take a break today!");
      } else {
        println!(
          "Today, run for {} minutes!",
          expensive_closure.value(intensity)
        );
      }
    }
  }
  generate_workout(10, 3)
}

calculating slowly...
Today, do 10 pushups!
Next, do 10 situps!


()

In [27]:
{
  fn cache_text(txt: &str) -> &str {
    let mut catcher = Catcher4::new(|s| {
      println!("calculating slowly...");
      thread::sleep(Duration::from_secs(2));
      s
    });

    catcher.value(txt)
  }

  cache_text(&String::from("hello"))
}

calculating slowly...


"hello"